In [141]:
#from tensorflow_docs.vis import embed
from tensorflow import keras
#from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
#import imageio
import cv2
import os
import json

Data preparation

Preparing training data

In [143]:
data_df = pd. read_csv('/workspaces/final-project-shot-type/data/CSV/dataset_df.csv', dtype={"shot": str})

data_df.drop(columns=['scale_val'], inplace=True)
data_df.drop(columns = ['move_val'], inplace=True)

train_path = '/workspaces/final-project-shot-type/data/shot-type-dataset/trailer_v3/train/'
data_df['video_name'] = train_path + 'shot_' + data_df['movie'] + '_' + data_df['shot'] + '.mp4'
train_df = data_df[data_df['dataset']=='train']
train_df = train_df[['video_name', 'move_label','scale_label']]
train_df = train_df.rename(columns ={'scale_label': 'tag'})


test_path = '/workspaces/final-project-shot-type/data/shot-type-dataset/trailer_v3/test/'
data_df['video_name'] = test_path + 'shot_' + data_df['movie'] + '_' + data_df['shot'] + '.mp4'
test_df = data_df[data_df['dataset']=='test']
test_df = test_df[['video_name', 'move_label','scale_label']]
test_df = test_df.rename(columns ={'scale_label': 'tag'})

In [144]:
train_df

,video_name,move_label,tag
0,/workspaces/final-project-shot-type/data/shot-...,Static,CS
1,/workspaces/final-project-shot-type/data/shot-...,Static,CS
2,/workspaces/final-project-shot-type/data/shot-...,Static,ECS
3,/workspaces/final-project-shot-type/data/shot-...,Static,MS
4,/workspaces/final-project-shot-type/data/shot-...,Static,CS
...,...,...,...
20851,/workspaces/final-project-shot-type/data/shot-...,Push,FS
20852,/workspaces/final-project-shot-type/data/shot-...,Motion,LS
20853,/workspaces/final-project-shot-type/data/shot-...,Push,LS
20854,/workspaces/final-project-shot-type/data/shot-...,Motion,FS


In [145]:
test_df

,video_name,move_label,tag
25466,/workspaces/final-project-shot-type/data/shot-...,Static,LS
25467,/workspaces/final-project-shot-type/data/shot-...,Static,MS
25468,/workspaces/final-project-shot-type/data/shot-...,Static,MS
25469,/workspaces/final-project-shot-type/data/shot-...,Static,LS
25470,/workspaces/final-project-shot-type/data/shot-...,Motion,FS
...,...,...,...
33648,/workspaces/final-project-shot-type/data/shot-...,Static,FS
33649,/workspaces/final-project-shot-type/data/shot-...,Motion,MS
33650,/workspaces/final-project-shot-type/data/shot-...,Motion,FS
33651,/workspaces/final-project-shot-type/data/shot-...,Static,LS


In [32]:
#df_train.to_csv('/workspaces/final-project-shot-type/data/CSV/dataset_train_v1.csv')

Preparing Test Data

In [41]:
#df_test.to_csv('/workspaces/final-project-shot-type/data/CSV/dataset_test_v1.csv')

In [146]:
train_df= train_df[0:10000]

# train_df = pd.read_csv('/workspaces/final-project-shot-type/data/CSV/dataset_train_v1.csv')
test_df = test_df[0:3600]

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 10000
Total videos for testing: 3600


,video_name,move_label,tag
3156,/workspaces/final-project-shot-type/data/shot-...,Static,ECS
5691,/workspaces/final-project-shot-type/data/shot-...,Motion,LS
9706,/workspaces/final-project-shot-type/data/shot-...,Static,FS
4834,/workspaces/final-project-shot-type/data/shot-...,Static,MS
923,/workspaces/final-project-shot-type/data/shot-...,Motion,CS
9914,/workspaces/final-project-shot-type/data/shot-...,Static,MS
9338,/workspaces/final-project-shot-type/data/shot-...,Static,CS
8806,/workspaces/final-project-shot-type/data/shot-...,Static,LS
7390,/workspaces/final-project-shot-type/data/shot-...,Static,LS
5902,/workspaces/final-project-shot-type/data/shot-...,Static,MS


In [147]:
train_df['tag'].value_counts()

CS     2230
MS     2204
ECS    2022
FS     1978
LS     1566
Name: tag, dtype: int64

Define hyperparameters

In [148]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 50

MAX_SEQ_LENGTH = 10
NUM_FEATURES = 2048

Loading frames and working on frame's size

In [150]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

Feature extractor

In [151]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()
print(feature_extractor.summary())

Model: "feature_extractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv_4 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_4 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
_________________________________________________________________
None


Working on labels

In [152]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['CS', 'ECS', 'FS', 'LS', 'MS']


Preparing videos

In [153]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

Frame features in train set: (10000, 10, 2048)
Frame masks in train set: (10000, 10)


Saving embeddings

Train

In [176]:
train_data_embedding_0_10k = np.save('/workspaces/final-project-shot-type/embeddings/train_data_embedding_0_10k.npy', train_data[0])

In [177]:
train_data_embedding_1_10k = np.save('/workspaces/final-project-shot-type/embeddings/train_data_embedding_1_10k.npy', train_data[1])

In [178]:
train_labels_embedding_10k = np.save('/workspaces/final-project-shot-type/embeddings/train_labels_embedding_10k.npy', train_labels)

Test

In [182]:
test_data_embedding_0_10k = np.save('/workspaces/final-project-shot-type/embeddings/test_data_embedding_0_10k.npy', test_data[0])

In [183]:
test_data_embedding_1_10k = np.save('/workspaces/final-project-shot-type/embeddings/test_data_embedding_1_10k.npy', test_data[1])

In [184]:
test_labels_embedding_10k = np.save('/workspaces/final-project-shot-type/embeddings/test_labels_embedding_10k.npy', test_labels)

Sequence model (RNN)

In [158]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"],
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/workspaces/final-project-shot-type/experiments/inceptionv3_GRU/exp_005/model.weights_exp_005.h5"
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=2, min_lr=0.000001)
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint, reduce_lr],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/50
218/219 [============================>.] - ETA: 0s - loss: 1.5007 - accuracy: 0.3019
Epoch 1: val_loss improved from inf to 1.33294, saving model to /workspaces/final-project-shot-type/experiments/inceptionv3_GRU/exp_005/model.weights_exp_005.h5
219/219 [==============================] - 23s 47ms/step - loss: 1.5002 - accuracy: 0.3024 - val_loss: 1.3329 - val_accuracy: 0.4460 - lr: 1.0000e-04
Epoch 2/50
218/219 [============================>.] - ETA: 0s - loss: 1.3192 - accuracy: 0.4167
Epoch 2: val_loss improved from 1.33294 to 1.20140, saving model to /workspaces/final-project-shot-type/experiments/inceptionv3_GRU/exp_005/model.weights_exp_005.h5
219/219 [==============================] - 5s 24ms/step - loss: 1.3192 - accuracy: 0.4167 - val_loss: 1.2014 - val_accuracy: 0.5110 - lr: 1.0000e-04
Epoch 3/50
219/219 [==============================] - ETA: 0s - loss: 1.2210 - accuracy: 0.4667
Epoch 3: val_loss improved from 1.20140 to 1.12294, saving model to /workspaces/final-p

Predictions

In [ ]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
# def to_gif(images):
#     converted_images = images.astype(np.uint8)
#     imageio.mimsave("animation.gif", converted_images, fps=10)
#     return embed.embed_file("animation.gif")


test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
# to_gif(test_frames[:MAX_SEQ_LENGTH])